In [1]:
# implemented and written by Yeoreum Lee in AI HnV Lab @ Sahmyook University in 2023
__author__ = 'leeyeoreum02'

In [2]:
from typing import Tuple, Callable

import numpy as np

In [3]:
x_data = np.array([[2, 4], [4, 11], [6, 6], [8, 5], [10, 7], [12, 16], [14, 8], [16, 3], [18, 7]])
t_data = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1]).reshape(9, 1)

print(x_data.shape, t_data.shape)

(9, 2) (9, 1)


### 1. 데이터 나누기(split)


In [4]:
def split_data(x_data: np.ndarray, t_data: np.ndarray, split_rate: float) -> Tuple[np.ndarray]:
    test_x_data = x_data[:int(split_rate * len(x_data))]
    test_t_data = t_data[:int(split_rate * len(t_data))]
    train_x_data = x_data[int(split_rate * len(x_data)):]
    train_t_data = t_data[int(split_rate * len(t_data)):]
    
    return train_x_data, train_t_data, test_x_data, test_t_data


train_x_data, train_t_data, test_x_data, test_t_data = split_data(x_data, t_data, split_rate=0.2)
print(train_x_data.shape, train_t_data.shape, test_x_data.shape, test_t_data.shape,)

(8, 2) (8, 1) (1, 2) (1, 1)


### 2. 활성 함수(activation function)

주어진 수식만을 이용하여 활성 함수 중 하나인 sigmoid 함수를 구현하시오. (구글링, 메인 교재 참고, 서브 강의 참고 금지)

$$sigmoid(\boldsymbol{x}) = \frac {1} {1 + e^{-\boldsymbol{x}}}$$

In [5]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

### 3. 로지스틱 회귀(logistic regression) 모델

메인 교재와 서브 강의만을 이용하여 로지스틱 회귀(logistic regression) 모델을 구현하시오. (구글링 금지)

$$\boldsymbol{y} = f(W, b)(\boldsymbol{x}) = sigmoid(W\boldsymbol{x} + b)$$

In [6]:
class LogisticRegression:
    def __init__(self, n_input: int, n_output: int) -> None:
        self.W = np.random.rand(n_input, n_output)
        self.b = np.random.rand(n_output)
        
    def forward(self, x: np.ndarray) -> np.ndarray:
        z = x @ self.W + self.b
        y = a = sigmoid(z)
        return y

    def __call__(self, x: np.ndarray) -> np.ndarray:
        return self.forward(x)
    
    
model = LogisticRegression(n_input=2, n_output=1)

### 4. 오차 함수 (error function, loss function)

주어진 수식만을 이용하여 CE(cross entropy) 오차 함수를 구현하시오. (구글링, 메인 교재 참고, 서브 강의 참고 금지)
- delta는 log의 진수 조건을 만족하기 위해 필요하므로 반드시 사용
- N은 데이터 개수 (행 개수)
- $y$는 정답(label) $\hat{y}$은 예측값(prediction)

$$CE = -\sum_{i=1} ^N (\boldsymbol{y_{i}} \cdot \log \boldsymbol{\hat{y_{i}}} + (1 - \boldsymbol{y_{i}}) \cdot \log (1 - \boldsymbol{\hat{y_{i}}}))$$

In [7]:
def cross_entropy_loss(y_data: np.ndarray, t_data: np.ndarray) -> np.ndarray:
    delta = 1e-4
    return -np.sum(t_data * np.log(y_data + delta) + (1 - t_data) * np.log((1 - y_data) + delta))

### 5. 편미분 함수 (partial numerical derivative)

메인 교재와 서브 강의만을을 이용하여 야코비안 행렬(jacobian matrix)을 반환하는 편미분(partial numerical derivative) 함수를 구현하시오. (구글링 금지)

$$J(W) = \frac{dE} {dW} = \begin{pmatrix} \frac{\partial E} {\partial W_{11}} & \frac {\partial E} {\partial W_{12}} \end{pmatrix}$$

$$J(\boldsymbol{b}) = \frac{dE} {d\boldsymbol{b}} = \begin{pmatrix} \frac{\partial E} {\partial b_{1}} \end{pmatrix}$$

In [8]:
def numerical_derivative(f: Callable, x: np.ndarray) -> np.ndarray:
    h = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        
        temp = x[idx]
        x[idx] = float(temp) + h
        fx1 = f(x)
        
        x[idx] = temp - h
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2 * h)
        
        x[idx] = temp
        it.iternext()
        
    return grad

### 6. 모델 학습 (train)

메인 교재와 서브 강의만을 이용하여 다음 순서를 가지는 학습 코드를 구현하시오. (구글링 금지)

1. 모델 순전파 (forward)
2. 오차 계산 (loss)
3. 모델 파라미터(가중치 + 편향) 별 오차 함수의 편미분값 계산 (numerical derivative)
4. 가중치(weight), 편향(bias) 갱신 (경사 하강법, gradient descent)

In [11]:
def train() -> None:
    lr = 1e-2
    
    for epoch in range(50000):
        y_data = model(train_x_data)
        loss = cross_entropy_loss(y_data, train_t_data)

        f = lambda W, b: sigmoid(train_x_data @ W + b)  # y = f(W, b)(x) = sigmoid(Wx + b)
        
        E_w = lambda W: cross_entropy_loss(f(W, model.b), train_t_data)
        E_b = lambda b: cross_entropy_loss(f(model.W, b), train_t_data)
        
        model.W -= lr * numerical_derivative(E_w, model.W)
        model.b -= lr * numerical_derivative(E_b, model.b)
        
        print(f'Epoch: {epoch}, loss {loss}')


train()

Epoch: 0, loss 0.11299510713865304
Epoch: 1, loss 0.11299302473715493
Epoch: 2, loss 0.11299094241115012
Epoch: 3, loss 0.112988860160635
Epoch: 4, loss 0.1129867779856049
Epoch: 5, loss 0.11298469588605668
Epoch: 6, loss 0.11298261386198509
Epoch: 7, loss 0.11298053191338711
Epoch: 8, loss 0.1129784500402576
Epoch: 9, loss 0.11297636824259394
Epoch: 10, loss 0.11297428652039099
Epoch: 11, loss 0.11297220487364408
Epoch: 12, loss 0.11297012330235004
Epoch: 13, loss 0.11296804180650524
Epoch: 14, loss 0.11296596038610479
Epoch: 15, loss 0.11296387904114502
Epoch: 16, loss 0.11296179777162146
Epoch: 17, loss 0.11295971657753097
Epoch: 18, loss 0.11295763545886811
Epoch: 19, loss 0.11295555441563004
Epoch: 20, loss 0.1129534734478119
Epoch: 21, loss 0.11295139255541005
Epoch: 22, loss 0.11294931173841902
Epoch: 23, loss 0.11294723099683782
Epoch: 24, loss 0.11294515033065955
Epoch: 25, loss 0.11294306973988129
Epoch: 26, loss 0.11294098922449888
Epoch: 27, loss 0.1129389087845078
Epoch: 2

### 7. 모델 추론 (evaluate)

메인 교재와 서브 강의만을 이용하여 추론 코드를 구현하시오. (구글링 금지)

In [10]:
def test(model=model):
    y_data = model(test_x_data)
    print(y_data, test_t_data)
    
    
test()

[[1.35419105e-07]] [[0]]
